In [13]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import fsolve, minimize, Bounds, LinearConstraint
from scipy.integrate import quad
# from scipy import linspace, meshgrid, arange, empty, concatenate, newaxis, shape
import matplotlib.pyplot as plt
from collections import deque
import time

In [14]:
def generate_service_time(x):
    # Exponentially distributed
    service_time = np.random.exponential(1/x)
    # Gamma distributed
    # service_times = np.random.gamma(x[0], x[1]) # Mean = x[0]x[1], Variance = x[0]x[1]^2
    return service_time

In [16]:
# Earlier approach

# def F(x, w, price, model_vars):
#     lam = model_vars[0]
#     theta_1 = model_vars[1]
#     theta_2 = model_vars[2]
#     if x >= w:
#         return 1 - np.exp(-lam*np.exp(-theta_1*price)*((1-np.exp(-theta_2*w))/theta_2 + (x-w)))
#     else:
#         return 1 - np.exp(-lam*np.exp(-theta_1*price)*(np.exp(-theta_2*(w-x))-np.exp(-theta_2*w))/theta_2)
    
    
# def inverse_F(zeta, w, price, model_vars):
#     lam = model_vars[0]
#     theta_1 = model_vars[1]
#     theta_2 = model_vars[2]
#     if zeta >= F(w, w, price, model_vars):
#         return w - (1-np.exp(-theta_2*w))/theta_2 - np.log(1-zeta)*np.exp(theta_1*price)/lam
#     else:
#         return (1/theta_2)*np.log(1-(theta_2/lam)*np.exp(theta_1*price+theta_2*w)*np.log(1-zeta))
    
    
# def gradient_inverse_F(zeta, w, gradient_w, price, model_vars):
#     lam = model_vars[0]
#     theta_1 = model_vars[1]
#     theta_2 = model_vars[2]
#     if zeta >= F(w, w, price, model_vars):
#         return (1 + np.exp(theta_2*w))*gradient_w - (theta_1/lam)*np.exp(theta_1*price)*np.log(1-zeta)
#     else:        
#         return (theta_1 + theta_2*gradient_w)/(theta_2 - lam*np.exp(-theta_1*price-theta_2*w)/np.log(1-zeta))


#################################################################################################################################

# New approach

def F(x, v, price, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    C_1 = 1.0
    C_2 = 1.0
    temp_const_1 = C_2 + theta_1* price**2
    if x >= v:
        # return 1 - np.exp(-lam*np.exp(-theta_1*price)*((1-np.exp(-theta_2*v))/theta_2 + (x-v)))
        var = (C_1/(np.sqrt(theta_2)*np.sqrt(temp_const_1))) * np.arctan(np.sqrt(theta_2)*v/np.sqrt(temp_const_1)) + C_1*(x-v)/temp_const_1
    else:
        # return 1 - np.exp(-lam*np.exp(-theta_1*price)*(np.exp(-theta_2*(v-x))-np.exp(-theta_2*v))/theta_2)
        var = (C_1/(np.sqrt(theta_2)*np.sqrt(temp_const_1))) * ( np.arctan(np.sqrt(theta_2)*v/np.sqrt(temp_const_1)) - np.arctan(np.sqrt(theta_2)*(v-x)/np.sqrt(temp_const_1)) )
    return 1 - np.exp(-lam*var)


    
def inverse_F(zeta, v, price, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    C_1 = 1.0
    C_2 = 1.0
    temp_const_1 = C_2 + theta_1* price**2
    if zeta >= F(v, v, price, model_vars):
        # return v - (1-np.exp(-theta_2*v))/theta_2 - np.log(1-zeta)*np.exp(theta_1*price)/lam
        return v - (temp_const_1/(C_1*lam))*np.log(1-zeta) - (np.sqrt(temp_const_1)/np.sqrt(theta_2))*np.arctan(np.sqrt(theta_2)*v/np.sqrt(temp_const_1))
    else:
        # return (1/theta_2)*np.log(1-(theta_2/lam)*np.exp(theta_1*price+theta_2*v)*np.log(1-zeta))
        return v - (np.sqrt(temp_const_1)/np.sqrt(theta_2)) * np.tan( np.arctan(np.sqrt(theta_2)*v/np.sqrt(temp_const_1)) + np.sqrt(theta_2)*np.sqrt(temp_const_1)/(C_1*lam) * np.log(1-zeta) )
    

    
def gradient_inverse_F(zeta, v, gradient_v, price, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    C_1 = 1.0
    C_2 = 1.0
    temp_const_1 = C_2 + theta_1* price**2
    if zeta >= F(v, v, price, model_vars):
        # return (1 - np.exp(-theta_2*v))*gradient_v - (theta_1/lam)*np.exp(theta_1*price)*np.log(1-zeta)
        return -2*theta_1*price/(C_1*lam) * np.log(1-zeta) + theta_1*price*v/(C_2 + theta_1*price**2 + theta_2*v**2) \
                    -theta_1*price/(np.sqrt(theta_2)*np.sqrt(C_2 + theta_1*price**2)) * np.arctan(np.sqrt(theta_2)*v/np.sqrt(C_2 + theta_1*price**2))
    else:        
        # return (theta_1 + theta_2*gradient_v)/(theta_2 - lam*np.exp(-theta_1*price-theta_2*v)/np.log(1-zeta))
        temp_const_2 = np.arctan(np.sqrt(theta_2)*v/np.sqrt(C_2 + theta_1*price**2)) + np.sqrt(theta_2)*np.sqrt(C_2 + theta_1*price**2)/(C_1*lam) * np.log(1-zeta)
        return gradient_v - theta_1*price/(np.sqrt(theta_2)*np.sqrt(C_2 + theta_1*price**2)) * np.tan(temp_const_2) - (1/np.cos(temp_const_2)**2) \
                    * (np.sqrt(theta_2)*np.sqrt(C_2 + theta_1*price**2)/(C_1*lam)) * ( theta_1*price*v/(C_2 + theta_1*price**2 + theta_2*v**2) + gradient_v + theta_1*price/(lam*C_1) * np.log(1-zeta) )

In [17]:
def queue_simulation(initial_workload, window_size, model_vars):
    workload = initial_workload

    time = 0
    
    w_plus = [0]
    w_minus = [0]
    interarrival_times = []
    service_times = []
    zetas = []
    prices = []
    
    while time < window_size:
        # Simulate interarrival time, service time
        price = np.random.uniform(10, 30)
        zeta = np.random.uniform(0, 1)
        next_interarrival_time = inverse_F(zeta, workload, price, model_vars)
        service_time = generate_service_time(1)
        # Store values for reference
        prices.append(price)
        zetas.append(zeta)
        interarrival_times.append(next_interarrival_time)
        service_times.append(service_time)
        w_minus.append(max(0, workload - next_interarrival_time))
        workload = max(0, workload - next_interarrival_time) + service_time
        w_plus.append(workload)
        # Update time
        time += next_interarrival_time

    # Suppose we simulated N effective arrivals, then w_minus, w_plus is of length N
    # Stores values corresponding to arrival 1, 2, 3, ..., N
    # It does not store workload values which simulated first arrival
    w_minus = np.array(w_minus)
    w_plus = np.array(w_plus)
    interarrival_times = np.array(interarrival_times)
    service_times = np.array(service_times)
    zetas = np.array(zetas)
    prices = np.array(prices)
    
    return [w_minus, w_plus, interarrival_times, service_times, zetas, prices]

In [18]:
# def Log_Likelihood(model_vars, w_plus, interarrival_times, price):
#     lam = model_vars[0]
#     theta_1 = model_vars[1]
#     theta_2 = model_vars[2]
    
#     N = np.size(interarrival_times)
    
#     temp_1 = np.maximum(0, w_plus[0:-1] - interarrival_times)
#     temp_2 = np.maximum(0, interarrival_times - w_plus[0:-1])
    
#     L = N * np.log(lam) - theta_1 * np.sum(price) - theta_2 * np.sum(temp_1) \
#         - lam * np.sum( np.exp(-theta_1*price) * ((np.exp(-theta_2*temp_1) - np.exp(-theta_2*w_plus[0:-1]))/theta_2 + temp_2) )
    
#     return -L

##############################################################################################################################################

def Log_Likelihood(model_vars, w_plus, interarrival_times, price):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    
    N = np.size(interarrival_times)
    
    temp_1 = np.maximum(0, w_plus[0:-1] - interarrival_times)
    
    C_1 = 1.0
    C_2 = 1.0
    # Fix: Use np.mean(price**2) instead of price**2 to get a scalar
    temp_const_1 = C_2 + theta_1 * np.mean(price**2)

    L = N * np.log(lam) - np.sum(np.log(C_2 + theta_1*price**2 + theta_2*temp_1**2))
    for i in range(N):
        if interarrival_times[i] >= w_plus[i]:
            var = (C_1/(np.sqrt(theta_2)*np.sqrt(temp_const_1))) * np.arctan(np.sqrt(theta_2)*w_plus[i]/np.sqrt(temp_const_1)) + C_1*(interarrival_times[i]-w_plus[i])/temp_const_1
        else:
            var = (C_1/(np.sqrt(theta_2)*np.sqrt(temp_const_1))) * ( np.arctan(np.sqrt(theta_2)*w_plus[i]/np.sqrt(temp_const_1)) - np.arctan(np.sqrt(theta_2)*(w_plus[i]-interarrival_times[i])/np.sqrt(temp_const_1)) )
        L -= lam * var

    return -L  # Make sure this returns a scalar

In [20]:
# Simulation

true_model_vars = [20, 0.1, 0.2]
initial_model_vars = [30, 0.5, 0.5]
initial_workload = 0
initial_window_size = 1000

[w_minus, w_plus, interarrival_times, service_times, zetas, prices] = queue_simulation(initial_workload, initial_window_size, true_model_vars)

estd_pars = minimize(Log_Likelihood, initial_model_vars, method = "L-BFGS-B", args = (w_plus, interarrival_times, prices), bounds = [(1, 100), (0.001, 0.999), (0.001, 0.999)])
estd_model_vars = estd_pars.x
print(estd_model_vars)

[1.0000000e+02 5.0427762e-01 1.0000000e-03]
